In [ ]:
# dependencies go here
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
from config import pw, gkey
import os

## set up database

We set up the `minneapolis_housing` database and related tables in Postgresql, using SQLalchemy:

* `neighborhood`
* `home_value`
* `crime`

Code for setting up the tables directly in Postgres can be found in the queries.sql file accompanying this notebook.

In [ ]:
conn = f"postgres:{pw}@localhost:5432/minneapolis_housing"
engine = create_engine(f'postgresql://{conn}')

In [ ]:
Base = declarative_base()

In [ ]:
# Drop tables if they exist
# Neighborhood.__table__.drop(engine)
# Crime.__table__.drop(engine)
# Home_Value.__table__.drop(engine)
# OR 
# Base.metadata.drop_all(engine)

In [ ]:
# Create base for later creating neighborhood table
class Neighborhood(Base):
    __tablename__ = 'neighborhood'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String(255))
    population_2010 = Column(Integer)

In [ ]:
# Create base for later creating crime table
class Crime(Base):
    __tablename__ = 'crime'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood_id = Column(Integer)
    crime_description = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)

In [ ]:
class Home_Value(Base):
    __tablename__ = 'home_value'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood_id = Column(Integer)
    property_type = Column(String(255))
    num_bedrooms = Column(Integer)
    sq_footage_house = Column(Integer)
    sq_footage_parcel = Column(Integer)
    address = Column(String(255))
    landuse = Column(String(255))
    value_total = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)

## load data
### Neighborhood datasets

In [ ]:
# Load Minneapolis city dataset for neighborhoods
n_path = os.path.join('.', 'data', 'Minneapolis_Neighborhoods.csv')
neighborhood = pd.read_csv(n_path)
neighborhood_df = neighborhood[['FID', 'BDNAME']].copy()
neighborhood_df.rename(columns={
    'FID': 'id',
    'BDNAME': 'neighborhood'
}, inplace=True)

'''
NOTES:
Noticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact
that it is a unique ID.
'''

In [ ]:
# Load Minneapolis neighborhood census dataset 
census_path = os.path.join('.', 'data', 'census_2010.xls')
census = pd.read_excel(census_path, header=None)
census_df = census.iloc[6:, :2]
census_df.rename(columns={
    0: 'neighborhood',
    1: 'population_2010'
}, inplace=True)

In [ ]:
# merge the dfs together into one neighborhood df
neighborhoods = neighborhood_df.merge(census_df, how='outer', on='neighborhood')

In [ ]:
# Check dataframe for null values
mask = pd.isnull(neighborhoods.id)
print(neighborhoods[mask])

mask = pd.isnull(neighborhoods.population_2010)
print(neighborhoods[mask])

# No population data for South Uptown (44) or Kenwood (74)
# No ids for CARAG (87) or Kenwood (88)

In [ ]:
# deal with null values 

# Kenwood (74) should be updated with Kenwood (88)'s pop data 
neighborhoods.loc[73, 'population_2010'] = neighborhoods.loc[88, 'population_2010']
neighborhoods.tail(20)

# create id for CARAG (87)
neighborhoods.loc[87, 'id'] = '88'

# Drop rows 88, 89
neighborhoods.drop([88,89], inplace=True)

In [ ]:
# Jenna start
#Load Assessors Parcel Data 2019 CSV & Create dataframe
csv_file = "data/Assessors_Parcel_Data_2019.csv"
assessors_df = pd.read_csv(csv_file)
assessors_df.head()

In [ ]:
#Create a new dataframe with select columns
assessors_df = assessors_df[['NEIGHBORHOOD', 
                                    'FORMATTED_ADDRESS',
                                   'PROPERTY_TYPE',
                                   'LANDUSE',
                                   'TOTALVALUE',
                                   'BELOWGROUNDAREA',
                                   'ABOVEGROUNDAREA',
                                   'BEDROOMS',
                                   'PARCEL_AREA_SQFT',
                                   'X',
                                   'Y']]
assessors_df.head()

In [ ]:
#Rename columns
assessors_parcel_df = assessors_df.rename(columns={"NEIGHBORHOOD":"neighborhood",
                                                   "FORMATTED_ADDRESS": "address",
                                                    "PROPERTY_TYPE": "property_type",
                                                    "LANDUSE": "landuse",
                                                    "TOTALVALUE":"value_total",
                                                    "BELOWGROUNDAREA": "below_grade_sq_footage",
                                                    "ABOVEGROUNDAREA": "above_grade_sq_footage",
                                                    "BEDROOMS": "num_bedrooms",
                                                    "PARCEL_AREA_SQFT": "sq_footage_parcel",
                                                    "X": "x",
                                                    "Y": "y"})

In [ ]:
#Filter out landuse categories that are not residential
residential_assessors_df = assessors_parcel_df[(assessors_parcel_df.landuse =='SINGLE-FAMILY DETACHED DWELLING')
                            |(assessors_parcel_df.landuse =='SINGLE-FAMILY ATTACHED DWELLING')
                            |(assessors_parcel_df.landuse =='MULTI-FAMILY RESIDENTIAL')
                            |(assessors_parcel_df.landuse =='MULTI-FAMILY APARTMENT')]

In [ ]:
#Filter out property types categories that are not residential
residential_df = residential_assessors_df[(residential_assessors_df.property_type =='Residential')
                |(residential_assessors_df.property_type =='Condominium')
                |(residential_assessors_df.property_type =='Double Bungalow')
                |(residential_assessors_df.property_type =='Apartment')
                |(residential_assessors_df.property_type =='Cooperative')
                |(residential_assessors_df.property_type =='Townhouse')
                |(residential_assessors_df.property_type =='Triplex')
                |(residential_assessors_df.property_type =='Residential - Zero Lot Line- DB')
                |(residential_assessors_df.property_type =='Residential Lake Shore')]                

residential_df.head()

In [ ]:
residential_df.tail()

In [ ]:
# Get lat/lng data based on address - planned to do this but it is cost prohibitive given the $0 budget for this
# project and the cost of querying the Google Geocoding API for ~130,000 addresses. 

In [ ]:
# Jenna end

In [ ]:
# Katrina start
#Loading data for police incidents and creating a dataframe for the data
csv_file = "data/Police_Incidents_2019.csv"
police_incidents_df = pd.read_csv(csv_file)
police_incidents_df.head()

In [ ]:
# Katrina end
#Create new dataframe with certain columns
incidents_df = police_incidents_df[['description',
                                    'neighborhood',
                                    'centerLat',
                                    'centerLong'
                                    ]]
incidents_df
incidents_df.rename(columns = {'centerLat':"Latitude",'centerLong':"Longitude"},inplace = True)
incidents_df

## transforming data

Things to transform all the data
1. Lower-case the neighborhood names on both dfs
2. Merge dfs on neighborhood names (into new crime df)
3. Drop neighborhood name from new crime df
4. Do same merge and neighborhood drop on assessor df

In [ ]:
# Stacy start

In [ ]:
neighborhoods.neighborhood = neighborhoods.neighborhood.str.lower()

In [ ]:
# Stacy end

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start
incidents_df.neighborhood = incidents_df.neighborhood.str.lower()
incidents_df

In [ ]:
# Katrina end
#join crimes table with neighborhood to get neighborhood ids in crime table
#use neighborhoods dataframe at the top of the notebook, neighborhoods is inner join
merge_table = pd.merge(neighborhoods, incidents_df, on = "neighborhood", how = "outer")
merge_table

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# load neighborhoods df into database table
neighborhoods.to_sql(name='neighborhood', con=engine, if_exists='replace', index=False)

In [ ]:
# Check that data is loaded into postgres
pd.read_sql_query('select * from neighborhood', con=engine).head()

In [ ]:
# Stacy end

In [ ]:
## Pulling it all together in an SQLalchemy query/pandas df for aggregate table